<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/v1.3.0/util/executionAnalytics/Notebooks/sevenBridgesCostAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install sevenbridges-python

In [ ]:
import sevenbridges as sbg

In [ ]:
token=''

In [ ]:
api = sbg.Api(url='https://cgc-api.sbgenomics.com/v2', token=token,advance_access=True)

In [ ]:
projectIds = []
for project in api.projects.query().all():
    projectIds.append(project.id)
projectIds

In [ ]:
import pandas as pd

data = []
for project in projectIds:
  for task in api.tasks.query(project=project, order_by='end_time',status='COMPLETED').all():
      data.append([task.id, project, task.batch])

df = pd.DataFrame(data, columns=['id','projectId', 'batch'])
df

In [ ]:
batch_tasks = df[df['batch'] == True].copy()
individual_tasks = df[df['batch'] == False].copy()
individual_tasks.loc[:, 'batchId'] = ''
individual_tasks.rename(columns={'id': 'taskId'}, inplace=True)
batch_tasks.rename(columns={'id': 'batchId'}, inplace=True)
individual_tasks

In [ ]:
batch_tasks_ids = batch_tasks['batchId'].tolist()
batch_data = []

for batchId in batch_tasks_ids:
    for task in api.tasks.query(order_by='end_time', parent=batchId, status='COMPLETED').all():
        batch_data.append([task.id, batchId ])

batch = pd.DataFrame(batch_data, columns=['taskId', 'batchId'])
batch_df = pd.merge(batch_tasks, batch, on='batchId', how='left').drop_duplicates().reset_index(drop=True)
batch_df

In [ ]:
final_task_df= pd.concat([batch_df,individual_tasks ])
final_task_df

In [ ]:
# task_ids = final_task_df['taskId'].tolist()

# data = []
# for task_id in task_ids:
#     task_details = api.tasks.get(id=task_id)
#     outputs = task_details.outputs
#     if isinstance(outputs, dict):
#         for key, value in outputs.items():
#             if value is not None and hasattr(value, 'id'):
#                 new_row = [
#                     task_details.id,
#                     task_details.name,
#                     task_details.status,
#                     task_details.project,
#                     task_details.app,
#                     task_details.type,
#                     task_details.batch,
#                     task_details.created_by,
#                     task_details.executed_by,
#                     task_details.start_time,
#                     task_details.end_time,
#                     task_details.price.amount,
#                     task_details.price.breakdown.storage,
#                     task_details.price.breakdown.computation,
#                     task_details.execution_status.steps_completed,
#                     task_details.execution_status.system_limit,
#                     task_details.execution_status.account_limit,
#                     task_details.execution_status.instance_init,
#                     task_details.execution_status.steps_total,
#                     task_details.execution_status.message,
#                     task_details.execution_status.duration,
#                     task_details.execution_status.queued_duration,
#                     task_details.execution_status.running_duration,
#                     task_details.execution_status.execution_duration,
#                     key,
#                     value.id
#                 ]
#                 data.append(new_row)

# details_df = pd.DataFrame(
#     data,
#     columns=[
#         'taskId',
#         'name',
#         'status',
#         'project',
#         'app',
#         'type',
#         'batch',
#         'created_by',
#         'executed_by',
#         'start_time',
#         'end_time',
#         'price',
#         'storage_price',
#         'compute_price',
#         'steps_completed',
#         'system_limit',
#         'account_limit',
#         'instance_init',
#         'steps_total',
#         'message',
#         'duration',
#         'queued_duration',
#         'running_duration',
#         'execution_duration',
#         'output_key',
#         'output_file_id'
#     ]
# )
# details_df

In [ ]:
# # Get the task IDs from the final task dataframe
# import time
# task_ids = final_task_df['taskId'].tolist()

# # Create an empty list to store the data
# data = []

# # Initialize a counter for the number of API requests made
# api_request_count = 0

# # Iterate over the task IDs
# for task_id in task_ids:

#     # Get the task details
#     task_details = api.tasks.get(id=task_id)

#     # Get the outputs
#     outputs = task_details.outputs

#     # Get the execution details
#     executionDetails= task_details.get_execution_details() 

#     # If the outputs are a dictionary
#     if isinstance(outputs, dict):

#         # Iterate over the keys and values in the outputs dictionary
#         for key, value in outputs.items():

#             # If the value is not None and has an ID
#             if value is not None and hasattr(value, 'id'):

#                 # Iterate over the jobs
#                 for job in executionDetails.jobs:
                
#                     # Create a new row for each job
#                     new_row = [
#                         task_details.id,
#                         task_details.name,
#                         task_details.status,
#                         task_details.project,
#                         task_details.app,
#                         task_details.type,
#                         task_details.batch,
#                         task_details.created_by,
#                         task_details.executed_by,
#                         task_details.start_time,
#                         task_details.end_time,
#                         task_details.price.amount,
#                         task_details.price.breakdown.storage,
#                         task_details.price.breakdown.computation,
#                         task_details.execution_status.steps_completed,
#                         task_details.execution_status.system_limit,
#                         task_details.execution_status.account_limit,
#                         task_details.execution_status.instance_init,
#                         task_details.execution_status.steps_total,
#                         task_details.execution_status.message,
#                         task_details.execution_status.duration,
#                         task_details.execution_status.queued_duration,
#                         task_details.execution_status.running_duration,
#                         task_details.execution_status.execution_duration,
#                         key,
#                         value.id,
#                         job.name,
#                         job.start_time,
#                         job.end_time,
#                         job.status,
#                         job.instance.id,
#                         job.instance.type,
#                         job.instance.provider,
#                         job.instance.disk.size,
#                         job.instance.disk.unit,
#                         job.instance.disk.type
#                     ]

#                     # Append the new row to the data list
#                     data.append(new_row)

#                     # Increment the API request counter
#                     api_request_count += 1

#                     # If we have made 500 API requests, wait for 5.5 minutes
#                     if api_request_count == 500:
#                         time.sleep(330)  # 330 seconds = 5.5 minutes
#                         api_request_count = 0  # reset the API request counter


# # Create a DataFrame from the data list
# details_df = pd.DataFrame(
#     data,
#     columns=[
#         'taskId',
#         'name',
#         'status',
#         'project',
#         'app',
#         'type',
#         'batch',
#         'created_by',
#         'executed_by',
#         'start_time',
#         'end_time',
#         'price',
#         'storage_price',
#         'compute_price',
#         'steps_completed',
#         'system_limit',
#         'account_limit',
#         'instance_init',
#         'steps_total',
#         'message',
#         'duration',
#         'queued_duration',
#         'running_duration',
#         'execution_duration',
#         'output_key',
#         'output_file_id',
#         'job_name',
#         'job_start_time',
#         'job_end_time',
#         'job_status',
#         'instance_id',
#         'instance_type',
#         'instance_provider',
#         'instance_disk_size',
#         'instance_disk_unit',
#         'instance_disk_type'
#     ]
# )
# details_df

In [ ]:
# Get the task IDs from the final task dataframe
import time
task_ids = final_task_df['taskId'].tolist()

# Create an empty list to store the data
data = []

# Initialize the API request count and time
api_request_count = 0
api_request_time = time.time()

# Iterate over the task IDs
for task_id in task_ids:

    # Get the task details
    task_details = api.tasks.get(id=task_id)
    api_request_count += 1

    # Get the outputs
    outputs = task_details.outputs
    api_request_count += 1

    # Get the execution details
    executionDetails= task_details.get_execution_details() 
    api_request_count += 1

    # Check if the API request count is a multiple of 500
    if api_request_count % 500 == 0:

        # Calculate the time elapsed since the last API request
        elapsed_time = time.time() - api_request_time

        # If less than 5.5 minutes have elapsed, sleep for the remaining time
        if elapsed_time < 330:
            time.sleep(330 - elapsed_time)

        # Reset the API request time
        api_request_time = time.time()

    # If the outputs are a dictionary
    if isinstance(outputs, dict):

        # Iterate over the keys and values in the outputs dictionary
        for key, value in outputs.items():

            # If the value is not None and has an ID
            if value is not None and hasattr(value, 'id'):

                # Iterate over the jobs
                for job in executionDetails.jobs:
                
                    # Create a new row for each job
                    new_row = [
                        task_details.id,
                        task_details.name,
                        task_details.status,
                        task_details.project,
                        task_details.app,
                        task_details.type,
                        task_details.batch,
                        task_details.created_by,
                        task_details.executed_by,
                        task_details.start_time,
                        task_details.end_time,
                        task_details.price.amount,
                        task_details.price.breakdown.storage,
                        task_details.price.breakdown.computation,
                        task_details.execution_status.steps_completed,
                        task_details.execution_status.system_limit,
                        task_details.execution_status.account_limit,
                        task_details.execution_status.instance_init,
                        task_details.execution_status.steps_total,
                        task_details.execution_status.message,
                        task_details.execution_status.duration,
                        task_details.execution_status.queued_duration,
                        task_details.execution_status.running_duration,
                        task_details.execution_status.execution_duration,
                        key,
                        value.id,
                        job.name,
                        job.start_time,
                        job.end_time,
                        job.status,
                        job.instance.id,
                        job.instance.type,
                        job.instance.provider,
                        job.instance.disk.size,
                        job.instance.disk.unit,
                        job.instance.disk.type
                    ]

                    # Append the new row to the data list
                    data.append(new_row)

# Create a DataFrame from the data list
details_df = pd.DataFrame(
    data,
    columns=[
        'taskId',
        'name',
        'status',
        'project',
        'app',
        'type',
        'batch',
        'created_by',
        'executed_by',
        'start_time',
        'end_time',
        'price',
        'storage_price',
        'compute_price',
        'steps_completed',
        'system_limit',
        'account_limit',
        'instance_init',
        'steps_total',
        'message',
        'duration',
        'queued_duration',
        'running_duration',
        'execution_duration',
        'output_key',
        'output_file_id',
        'job_name',
        'job_start_time',
        'job_end_time',
        'job_status',
        'instance_id',
        'instance_type',
        'instance_provider',
        'instance_disk_size',
        'instance_disk_unit',
        'instance_disk_type'
    ]
)
details_df

In [ ]:
final_detailed_task_df = pd.merge(final_task_df, details_df, on='taskId', how='left').drop_duplicates().reset_index(drop=True)
final_detailed_task_df

In [ ]:
import time

outputFiles = final_detailed_task_df['output_file_id'].to_list()
# Get unique list of output files
unique_output_files = list(set(outputFiles))
time.sleep(5.5 * 60)
# Iterate over unique list and populate dataframe
for file in unique_output_files:
    try:
        fileInfo = api.files.get(file)
        name = fileInfo.name
        url = fileInfo.download_info().url
        
        # Find all rows with the current file ID and populate with file name and URL
        final_detailed_task_df.loc[final_detailed_task_df['output_file_id'] == file, 'file_name'] = name
        final_detailed_task_df.loc[final_detailed_task_df['output_file_id'] == file, 'file_url'] = url
        
    except Exception as e:
        print(f'Error retrieving file {file}: {e}')
    
    if (unique_output_files.index(file) + 1) % 500 == 0:
        time.sleep(5.5 * 60)
final_detailed_task_df

In [ ]:
final_detailed_task_df

In [ ]:
final_detailed_task_df.to_csv('final.csv')